<a href="https://colab.research.google.com/github/mahjiid/mahjiid/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Oct  2 12:15:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'ABDULMAJEED OLASUNKANMI ABDULRASHEED.docx'
'ABDULRASHEED ABDULMAJEED.docx'
'Abdulrasheed abdulmajeed.pdf'
'Among all the hard things we have to do everyday, the most difficult one of them all is to keep going.gdoc'
 ATT00001
'Available Digital Freelancers.gsheet'
 Brokenft.gsheet
 Classroom
 CoinbaseWalletBackups
'Colab Notebooks'
 DriveFire
 Fiverr
 Flutter
'How to get started with Drive.pdf'
'JAMB Change of Course Institution - Nairaschools Services'
'laptop life style.pdf'
'My Drive'
'NURUDEEN WAEC.pdf'
 pass-1.jpg
'Project proposal.gdoc'
 Scanned_20190301-0331.pdf
'Session - 13 On 14-02-2018.MP4'
'Untitled document.gdoc'
 yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15494, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15494 (delta 0), reused 2 (delta 0), pack-reused 15490
Receiving objects: 100% (15494/15494), 14.11 MiB | 18.20 MiB/s, done.
Resolving deltas: 100% (10408/10408), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-10-02 12:17:01--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  94.8MB/s    in 1.6s    

2022-10-02 12:17:03 (94.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/david-clode-AF0CrhiVCMM-unsplash.jpg  
  inflating: data/obj/david-clode-AF0CrhiVCMM-unsplash.txt  
  inflating: data/obj/david-clode-CIMk0FSOrAE-unsplash.jpg  
  inflating: data/obj/david-clode-CIMk0FSOrAE-unsplash.txt  
  inflating: data/obj/david-clode-d8N22qmJEt4-unsplash.jpg  
  inflating: data/obj/david-clode-d8N22qmJEt4-unsplash.txt  
  inflating: data/obj/david-clode-J4RKji2HjmU-unsplash.jpg  
  inflating: data/obj/david-clode-J4RKji2HjmU-unsplash.txt  
  inflating: data/obj/david-clode-YQXwaRGamfs-unsplash.jpg  
  inflating: data/obj/david-clode-YQXwaRGamfs-unsplash.txt  
  inflating: data/obj/ellicia-4xNDhf0nA-Y-unsplash (1).jpg  
  inflating: data/obj/ellicia-4xNDhf0nA-Y-unsplash (1).txt  
  inflating: data/obj/ellicia-4xNDhf0nA-Y-unsplash.jpg  
  inflating: data/obj/ellicia-4xNDhf0nA-Y-unsplash.txt  
  inflating: data/obj/ellicia-VQrMbyxJNDw-unsplash.jpg  
  inflating: data/obj/

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.304204 0.396364 0.601814 0.649270

['0', '0.304204', '0.396364', '0.601814', '0.649270']
0 data/obj/pexels-valeriia-miller-5403361.txt
0 0.304204 0.396364 0.601814 0.649270
0 0.477141 0.490741 0.463927 0.685185

['0', '0.477141', '0.490741', '0.463927', '0.685185']
1 data/obj/pexels-bjørn-nielsen-7882793.txt
0 0.477141 0.490741 0.463927 0.685185
koala

[]
0 0.527284 0.573558 0.767067 0.493910

['0', '0.527284', '0.573558', '0.767067', '0.493910']
3 data/obj/mj-p43DM2gsc9o-unsplash.txt
0 0.527284 0.573558 0.767067 0.493910
0 0.461901 0.618315 0.815794 0.763369

['0', '0.461901', '0.618315', '0.815794', '0.763369']
4 data/obj/pexels-jonathan-lajoie-7442086.txt
0 0.461901 0.618315 0.815794 0.763369
0 0.521527 0.454778 0.956945 0.766000

['0', '0.521527', '0.454778', '0.956945', '0.766000']
5 data/obj/david-clode-d8N22qmJEt4-unsplash.txt
0 0.521527 0.454778 0.956945 0.766000
0 0.544831 0.530671 0.384045 0.695315

['0', '0.544831', '0.530671', '0.384045', '0.695315']
6 data/obj/ellicia-

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/david-clode-CIMk0FSOrAE-unsplash.jpg', 'data/obj/elmer-canas-YyG3TjhAgcg-unsplash.jpg', 'data/obj/pexels-pascal-renet-105823.jpg', 'data/obj/pexels-mehmet-turgut-kirkgoz-11922492.jpg', 'data/obj/pexels-mirko-ott-2942778.jpg', 'data/obj/max-zhang-aBDLySu9p_E-unsplash.jpg', 'data/obj/kerin-gedge-P3sEj29SHD8-unsplash.jpg', 'data/obj/gilles-rolland-monnet-XObb1sV-hns-unsplash.jpg', 'data/obj/laura-barry-DRImlWgq5u8-unsplash.jpg', 'data/obj/pexels-karen-laårk-boshoff-8395182.jpg', 'data/obj/pexels-marlon-trottmann-5843485.jpg', 'data/obj/axel-blanchard-y-rbyzpxInc-unsplash.jpg', 'data/obj/david-clode-J4RKji2HjmU-unsplash.jpg', 'data/obj/siem-van-woerkom-37nC8XgoIMM-unsplash.jpg', 'data/obj/pexels-bjørn-nielsen-7882793.jpg', 'data/obj/pexels-valeriia-miller-6090285.jpg', 'data/obj/geraldine-dukes-kr-_emGeNhM-unsplash.jpg', 'data/obj/bob-walker-wqyjvlpXwxc-unsplash.jpg', 'data/obj/steve-franklin-eI0m5qV7q5Y-unsplash.jpg', 'data/obj/adam-bignell-9bxmC5Gb20A-unsplash.jpg', 'data/obj/

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho